In [38]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\user\appdata\local\temp\pip-req-build-1a8z60so


  ERROR: Error [WinError 2] 지정된 파일을 찾을 수 없습니다 while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [35]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import transformers
import torch
import os
import nltk
import re

from tqdm import tqdm
tqdm.pandas()

In [40]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

ModuleNotFoundError: No module named 'kobert_tokenizer'

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
# KoBERT 모델과 토크나이저 불러오기
model_name = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [2]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

In [10]:
max_len = 512

In [26]:
def tokenize(text, max_length=max_len):
    tokens = tokenizer.encode_plus(text, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = tokens['input_ids'].to("cuda")
    attention_mask = tokens['attention_mask'].to("cuda")
    return input_ids, attention_mask

In [27]:
def get_embedding(input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
    last_hidden_states = outputs[0]
    last_4_layers = last_hidden_states[-4:]
    average_pooling = torch.mean(last_4_layers, dim=0)
    embedding = torch.sum(average_pooling, dim=0).numpy()
    return embedding

In [28]:
df = pd.read_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\관세용어사전_길이순.xlsx")

In [14]:
df_high = df[:200]
df_middle = df[200:400]
df_low = df[400:600]

In [30]:
input_ids, attention_mask = tokenize(a)
embedding = get_embedding(input_ids, attention_mask)
print(embedding)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
get_embedding()

In [16]:
kobert_embed_high = df_high['단어이름'].apply(lambda x: bert_embedding(x))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [15]:
kobert_embed_high = df_high['단어이름'].apply(lambda x: bert_embedding(x))
kobert_embed_middle = df_middle['단어이름'].apply(lambda x: bert_embedding(x))
kobert_embed_low = df_low['단어이름'].apply(lambda x: bert_embedding(x))

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import tqdm
import time
import numpy as np

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

In [ ]:
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')

In [ ]:
model.to('cuda')

In [ ]:
max_len = 512

In [ ]:
def tokenizer_(text, max_len, tokenizer):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    input_id = encoded_dict['input_ids']
    token_type_id = encoded_dict['token_type_ids']
    attention_mask = encoded_dict['attention_mask']
    
    return input_id, token_type_id, attention_mask

In [ ]:
def bert_embedding(text):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    tokens_tensor = torch.tensor([encoded_dict['input_ids']]).to('cuda')  
    segment_tensors = torch.tensor([encoded_dict['token_type_ids']]).to('cuda')
    attention_tensors = torch.tensor([encoded_dict['attention_mask']]).to('cuda')
    
    model.eval()

    with torch.no_grad():
        outputs = model(tokens_tensor, attention_tensors, segment_tensors)
        
    hidden_states = outputs[2]
    
    token_mean = []

    for h in hidden_states[-4:]:
        token_mean.append(torch.mean(h[0], dim=0))

    last_four_sentence_embedding = sum(token_mean)
    
    return last_four_sentence_embedding.cpu().numpy()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/아영/제주학회/관세용어사전_길이순.xlsx', index_col = 0)

In [ ]:
df

In [ ]:
len(df['단어 설명'])

In [ ]:
df[['단어 설명']]

In [ ]:
text = df["단어 설명"][0]

In [ ]:
text

In [ ]:
tokenizer(
    text,
    return_token_type_ids=False,
    return_attention_mask=False
)

In [ ]:
print(tokenizer.convert_ids_to_tokens([2, 3551, 5561, 6286, 517, 5330, 1828, 7815, 2906, 7138, 6875, 7095, 5174, 7088, 5033, 6723, 3686, 4756, 2683, 6230, 7095, 2802, 7942, 4511, 6723, 3883, 3551, 5561, 6286, 517, 6016, 2619, 2114, 7095, 4918, 7202, 3487, 7119, 517, 7088, 5146, 7828, 517, 6116, 3628, 7837, 3551, 517, 5760, 776, 5030, 6553, 6107, 5655, 867, 6553, 6107, 1815, 2019, 3551, 517, 6433, 7005, 7199, 6150, 7088, 4756, 4022, 6723, 3647, 2018, 5330, 3551, 2718, 6323, 6896, 3994, 5891, 2048, 867, 5330, 3551, 2718, 6323, 1434, 6903, 3676, 6855, 7327, 912, 4788, 3794, 517, 7901, 7096, 1633, 5907, 5904, 1835, 1233, 2724, 1103, 6926, 7096, 5125, 5887, 3860, 910, 3692, 2773, 7828, 2186, 6983, 834, 2724, 4892, 7095, 4049, 7202, 3765, 5330, 737, 7788, 3551, 517, 5330, 839, 5895, 5561, 1848, 3794, 1289, 5362, 6079, 2802, 7636, 6527, 7095, 4512, 6797, 6079, 5146, 5887, 3860, 3551, 517, 5859, 2724, 1103, 6926, 7095, 2802, 7942, 4511, 6723, 3860, 912, 2604, 3647, 3551, 517, 5760, 2724, 6879, 5337, 776, 1103, 6926, 6875, 6116, 3021, 7996, 4465, 7836, 2872, 3880, 2571, 3101, 776, 4104, 4130, 7789, 1849, 2724, 1103, 6926, 6485, 3101, 2683, 6230, 7088, 4511, 6723, 3883, 1370, 3551, 517, 5760, 2724, 1103, 6926, 7098, 2683, 2906, 7138, 6875, 6116, 3021, 7996, 4465, 7837, 3647, 3264, 6896, 517, 5330, 1720, 5888, 4012, 6900, 2121, 5330, 4022, 5889, 1706, 2396, 517, 7127, 889, 6072, 7075, 3650, 7096, 1720, 5899, 5176, 6900, 2962, 1434, 6896, 737, 7784, 517, 5895, 7089, 2121, 7095, 4022, 7096, 1096, 6615, 7078, 3676, 6855, 7344, 905, 7136, 517, 5760, 3758, 7720, 7005, 7720, 4064, 7622, 6055, 6664, 7700, 6705, 6267, 6122, 3438, 6190, 7126, 7088, 877, 839, 5899, 4480, 7229, 7637, 6629, 5788, 7096, 6254, 6079, 3658, 2724, 1103, 6926, 7098, 2683, 6629, 7468, 7138, 3264, 7418, 6896, 517, 7962, 5561, 7202, 1295, 6116, 4926, 3883, 3]))

In [ ]:
df['단어 설명']

In [ ]:
kobert_embed = df['단어 설명'].apply(lambda x: bert_embedding(x))